# doc2vec with gensim

This notebook demonstrates the basic commands needed to run doc2vec using the functions provided by by working through a toy example. This is using python 2.7 and gensim version 2.2.0.

Doc2vec is an extension of word2vec which converts documents (which could be any sequences of words, be they paragraphs, comments, or even whole documents). This notebook only demonstrates how to use the technique, it does not explain how it works. For details consult the paper by Le and Mikolov: https://arxiv.org/abs/1405.4053.

## Documents to vectors
In this first section we will show how to produce a vector for each document. First load the required packages.

In [1]:
import re
import numpy as np
import pandas as pd

from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

Next we need to define a couple of helper functions. One which splits ('tokenises') a string into a list of words, and one which takes puts our data into the data structure gensim is expecting.

In [2]:
# Tokenise on whitespace
def tokeniser(text):
    return re.findall(r"\S+", text)

# Create labelled sentence data structure
def labeller_doc(tag, text):
    return TaggedDocument(tokeniser(text), [tag])

Let's see what the tokeniser does:

In [3]:
tokeniser('an example string of text')

['an', 'example', 'string', 'of', 'text']

This is a very simple tokeniser, much more advanced ones are available. 

Doc2vec expects data in the 'TaggedDocument' format, which is a list of words within the documents and a list of IDs. In this case we are creating one vector per document so each document needs one, unique ID. For example:

In [4]:
ex_document = 'an example document'
ex_id = 'doc1'
labeller_doc(ex_id, ex_document)

TaggedDocument(words=['an', 'example', 'document'], tags=['doc1'])

Let's consider the example where we have text within a pandas dataframe.

In [5]:
# Create example data
df = pd.DataFrame()
df['text'] = [
                'this is text', 
                'any more text?', 
                'even more text!', 
                'words, all words'
            ]

We can use the dataframe IDs as the document IDs and convert all the documents into the required TaggedDocument format.

In [6]:
# Get data into correct format
documents =  df.apply(lambda x: labeller_doc(str(x.name), x['text']), axis=1).values

print documents

[TaggedDocument(words=['this', 'is', 'text'], tags=['0'])
 TaggedDocument(words=['any', 'more', 'text?'], tags=['1'])
 TaggedDocument(words=['even', 'more', 'text!'], tags=['2'])
 TaggedDocument(words=['words,', 'all', 'words'], tags=['3'])]


Then we can build the doc2vec model and delete some of the items created to free up memory.

In [7]:
# Build model
model = Doc2Vec(documents = documents,
        size = 2,
        seed = 42,
        min_count = 1,
        max_vocab_size = None,
        window = 5,
        iter = 5)

# Free up memory
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

We can retrieve vectors for each document:

In [8]:
model.docvecs.doctag_syn0

array([[-0.03483918, -0.16661185],
       [ 0.18773492,  0.22185083],
       [-0.12609929,  0.06258722],
       [ 0.11312576,  0.16608955]], dtype=float32)

And the IDs associated with each vector:

In [9]:
model.docvecs.doctags.keys()

['1', '0', '3', '2']

Note, this shows that the order of the vectors does not follow that of the original data frame. But you can retrieve the text associated with each of these vectors in the correct order:

In [10]:
df_new = pd.DataFrame()
df_new['text'] = df.loc[map(int, model.docvecs.doctags.keys()), 'text']
df_new

,text
1,any more text?
0,this is text
3,"words, all words"
2,even more text!


Vectors for new documents can be inferred:

In [11]:
new_sentence = ['one', 'more', 'sentence']
model.infer_vector(new_sentence)

array([-0.24704833,  0.23743619], dtype=float32)

As can the vectors for multiple sentences:

In [12]:
new_sentences = [ ['first', 'text'], ['second', 'test']]
np.matrix(map(model.infer_vector, new_sentences))

matrix([[ 0.07819276, -0.06925946],
        [ 0.01494676, -0.2055566 ]], dtype=float32)